<h1>20156042 Kim GaYoung</h1>

In [1]:
import numpy.linalg as lin

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random

file_data		= "mnist_train.csv"
handle_file	= open(file_data, "r")
data        		= handle_file.readlines()
handle_file.close()

size_row	= 28    # height of the image
size_col  	= 28    # width of the image

num_image	= len(data)
count       	= 0     # count for the number of images

#
# normalize the values of the input data to be [0, 1]
#
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

#
# example of distance function between two vectors x and y
#
def distance(x, y):

    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)

    return(s)

#
# make a matrix each column of which represents an images in a vector form 
#
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)

for line in data:

    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector)

    list_label[count]       = label
    list_image[:, count]    = im_vector    

    count += 1

In [3]:
# 테스트 데이터 불러오기
file_data_test		= "mnist_test.csv"

handle_file	= open(file_data_test, "r")
data_test      		= handle_file.readlines()
handle_file.close()

size_row	= 28    # height of the image
size_col  	= 28    # width of the image

num_image_test = len(data_test)
count       	= 0     # count for the number of images

list_image_test = np.empty((size_row * size_col, num_image_test), dtype=float)
list_label_test  = np.empty(num_image_test, dtype=int)
    
for line in data_test:
    line_data_test = line.split(',')
    label_test     = line_data_test[0]
    im_vector_test = np.asfarray(line_data_test[1:])
    im_vector_test = normalize(im_vector_test)
    
    list_label_test[count]   = label_test
    list_image_test[:,count]      = im_vector_test
    count += 1

1. Compute an optimal model parameter using the training dataset

In [6]:
A = np.zeros((num_image,size_row * size_col+1),dtype = 'float')      # A is data
B = np.zeros((num_image,1),dtype = 'float')                          # B is (-1 or 1)  ->  Ax = B

for i in range(num_image):
    A[i,0] = 1
    A[i,1:len(A[0,:])] = list_image[:,i]
    if list_label[i] == 0 :
        B[i] = 1
    else : 
        B[i] = -1
        
    for j in range(size_row * size_col):
        if list_image[j][i] == 0:
            list_image[j][i] += random.randrange(1,5)/255

In [7]:
# theta 구하기
theta = np.zeros((size_row * size_col+1,1), dtype = 'float')
theta = lin.inv(np.matrix(A.T) * np.matrix(A)) * np.matrix(A.T) * np.matrix(B)

In [13]:
#앞에 구한 theta를 이용해 test data로 B구하기 
testA = np.zeros((num_image_test, size_row * size_col+1),dtype = 'float')   #testA is data
testB = np.zeros((num_image_test, 1),dtype = 'float')                       #testB is (-1 or 1)  ->  Ax = B
for i in range(num_image_test):
    testA[i,0] = 1
    testA[i,1:len(testA[0,:])] = list_image_test[:,i]
    
testB = testA*theta

2. Compute (1) True Positive, (2) False Positive, (3) True Negative, (4) False Negative

- testing dataset

In [10]:
#testing dataset accuracy

count_zero = 0       #실제 0인 데이터 갯수

count_tpositive = 0  
count_tnegative = 0  
count_fpositive = 0
count_fnegative = 0

for i in range(num_image_test):
        
    if(list_label_test[i]==0):         #실제 0일때
        count_zero += 1
        if(testB[i] >= 0):             #내가 0이라고 대답한거
            count_tpositive += 1
        else:                          #내가 0이 아니라고 대답한거
            count_fnegative += 1
            
    else :                             #실제 0이 아닐때
        
        if(testB[i] < 0):              #내가 0이 아니라고 대답한거
            count_tnegative += 1
        else:                          #내가 0이라고 대답한거
            count_fpositive += 1
            
count_nonzero = num_image_test - count_zero #실제 0이 아닌 데이터 갯수

true_positive = count_tpositive / count_zero 
true_negative = count_tnegative / count_nonzero
false_positive = count_fpositive / count_nonzero
false_negative = count_fnegative / count_zero

accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)

print("true_positive  : " , true_positive)
print("true_negative  : " , true_negative)
print("false_positive : " , false_positive)
print("false_negative : " , false_negative)
print("accuracy : " , accuracy)

true_positive  :  0.9285714285714286
true_negative  :  0.992350332594235
false_positive :  0.0076496674057649665
false_negative :  0.07142857142857142
accuracy :  0.9604608805828319


- training dataset

In [12]:
# training dataset accuracy

count_zero = 0       #실제 0인 데이터 갯수

count_tpositive = 0  
count_tnegative = 0  
count_fpositive = 0
count_fnegative = 0

trainingB = A*theta

for i in range(num_image_test):
    if(list_label[i]==0):         #실제 0일때
        count_zero += 1
        if(trainingB[i] >= 0):             #내가 0이라고 대답한거
            count_tpositive += 1
        else:                          #내가 0이 아니라고 대답한거
            count_fnegative += 1
            
    else :                             #실제 0이 아닐때
        
        if(trainingB[i] < 0):              #내가 0이 아니라고 대답한거
            count_tnegative += 1
        else:                          #내가 0이라고 대답한거
            count_fpositive += 1
            
count_nonzero = num_image_test - count_zero #실제 0이 아닌 데이터 갯수

true_positive = count_tpositive / count_zero 
true_negative = count_tnegative / count_nonzero
false_positive = count_fpositive / count_nonzero
false_negative = count_fnegative / count_zero

accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)

print("true_positive  : " , true_positive)
print("true_negative  : " , true_negative)
print("false_positive : " , false_positive)
print("false_negative : " , false_negative)
print("accuracy : " , accuracy)

true_positive  :  0.8831168831168831
true_negative  :  0.9959995555061674
false_positive :  0.004000444493832648
false_negative :  0.11688311688311688
accuracy :  0.9395582193115253
